# Particle Filter for Nonlinear State Estimation

This notebook demonstrates the **Particle Filter (PF)**, also known as **Sequential Monte Carlo**, for nonlinear and non-Gaussian state estimation.

**When to Use Particle Filters**:
- Highly nonlinear dynamics or measurements
- Non-Gaussian noise (e.g., heavy-tailed, multimodal)
- Multi-hypothesis tracking (object could be in multiple locations)
- Discrete state spaces mixed with continuous

**Key Advantage**: Can represent arbitrary probability distributions using weighted particles.

**Reference**: Gordon et al. (1993), "Novel approach to nonlinear/non-Gaussian Bayesian state estimation"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pykal import DynamicalSystem
from pykal.algorithm_library.estimators.pf import PF

np.random.seed(42)

## 1. Problem Setup: Tracking with Bearing-Only Measurements

We'll track an object moving in 2D using only **bearing (angle) measurements**. This is a classic nonlinear estimation problem.

### Dynamics (Linear)
$$\mathbf{x}_k = \begin{bmatrix} x \\ y \\ \dot{x} \\ \dot{y} \end{bmatrix}_k = \mathbf{F} \mathbf{x}_{k-1} + \mathbf{w}_k$$

where $\mathbf{F}$ is constant velocity model.

### Measurement (Nonlinear)
$$y_k = \text{atan2}(y_k, x_k) + v_k$$

The **atan2** nonlinearity makes this challenging for EKF/UKF.

In [ ]:
# Simulation parameters
dt = 0.1  # Time step
T = 10.0  # Duration
t_sim = np.arange(0, T, dt)
N = len(t_sim)

# State transition matrix (constant velocity)
F = np.array([[1, 0, dt, 0],
              [0, 1, 0, dt],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

# Process noise
Q = np.diag([0.01, 0.01, 0.1, 0.1])

# Measurement noise (bearing angle)
R = np.array([[0.1**2]])  # 0.1 rad std dev

# Generate true trajectory (circular motion)
radius = 5.0
omega = 0.3  # Angular velocity
x_true = np.zeros((N, 4))

for k in range(N):
    angle = omega * t_sim[k]
    x_true[k, 0] = radius * np.cos(angle)  # x position
    x_true[k, 1] = radius * np.sin(angle)  # y position
    x_true[k, 2] = -radius * omega * np.sin(angle)  # x velocity
    x_true[k, 3] = radius * omega * np.cos(angle)   # y velocity

print(f"Simulating {N} timesteps over {T} seconds")
print(f"Object follows circular trajectory with radius {radius}m")

In [ ]:
# Generate bearing measurements
y_true = np.arctan2(x_true[:, 1], x_true[:, 0])
y_meas = y_true + np.random.normal(0, np.sqrt(R[0, 0]), N)

# Wrap angles to [-pi, pi]
y_meas = np.arctan2(np.sin(y_meas), np.cos(y_meas))

# Visualize trajectory and measurements
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Position
axes[0].plot(x_true[:, 0], x_true[:, 1], 'b-', linewidth=2, label='True trajectory')
axes[0].plot(0, 0, 'r*', markersize=15, label='Sensor at origin')
axes[0].set_xlabel('X position (m)')
axes[0].set_ylabel('Y position (m)')
axes[0].set_title('True Object Trajectory')
axes[0].legend()
axes[0].grid(True)
axes[0].axis('equal')

# Bearing measurements
axes[1].plot(t_sim, y_true, 'b-', linewidth=2, label='True bearing')
axes[1].plot(t_sim, y_meas, 'r.', markersize=4, label='Noisy measurements')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Bearing angle (rad)')
axes[1].set_title('Bearing Measurements')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 2. Particle Filter Setup

Define the dynamics and measurement functions for the particle filter.

In [ ]:
def f_dynamics(x):
    """State transition function with process noise"""
    return F @ x + np.random.multivariate_normal(np.zeros(4), Q)

def h_measurement(x):
    """Bearing measurement from position"""
    return np.array([np.arctan2(x[1], x[0])])

def likelihood(yk, x):
    """Measurement likelihood p(y_k | x_k)"""
    y_pred = h_measurement(x)
    
    # Angular difference (handle wrap-around)
    diff = yk[0] - y_pred[0]
    diff = np.arctan2(np.sin(diff), np.cos(diff))
    
    # Gaussian likelihood
    return np.exp(-0.5 * diff**2 / R[0, 0]) / np.sqrt(2 * np.pi * R[0, 0])

print("Particle filter functions defined")

## 3. Initialize Particle Filter

Create particles with initial distribution.

In [ ]:
# Particle filter parameters
N_particles = 500

# Initial particle distribution (uncertain position, small velocity)
particles_init = np.random.multivariate_normal(
    mean=np.array([3.0, 3.0, 0.0, 0.0]),  # Initial guess
    cov=np.diag([2.0, 2.0, 0.5, 0.5]),
    size=N_particles
)

weights_init = np.ones(N_particles) / N_particles

particles_weights = (particles_init, weights_init)

print(f"Initialized {N_particles} particles")
print(f"Initial mean estimate: {particles_init.mean(axis=0)[:2]}")
print(f"True initial position: {x_true[0, :2]}")

## 4. Run Particle Filter

Execute the particle filter over all measurements.

In [ ]:
# Storage for results
x_est_pf = []
particles_history = []
weights_history = []

# Run particle filter
for k in range(N):
    # Update step
    particles_weights = PF.f(
        particles_weights=particles_weights,
        yk=y_meas[k:k+1],
        f=f_dynamics,
        f_params={},
        likelihood=likelihood,
        likelihood_params={},
        resample_method='systematic',
        resample_threshold=0.5
    )
    
    # Extract estimate
    x_est = PF.h(particles_weights)
    x_est_pf.append(x_est)
    
    # Store for visualization
    particles_history.append(particles_weights[0].copy())
    weights_history.append(particles_weights[1].copy())

x_est_pf = np.array(x_est_pf)

print("\nParticle filter completed!")
print(f"Final position estimate: {x_est_pf[-1, :2]}")
print(f"True final position: {x_true[-1, :2]}")
print(f"Position error: {np.linalg.norm(x_est_pf[-1, :2] - x_true[-1, :2]):.3f} m")

## 5. Visualize Results

Compare particle filter estimates with ground truth.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Trajectory comparison
axes[0, 0].plot(x_true[:, 0], x_true[:, 1], 'b-', linewidth=2, label='True trajectory')
axes[0, 0].plot(x_est_pf[:, 0], x_est_pf[:, 1], 'r--', linewidth=2, label='PF estimate')
axes[0, 0].plot(0, 0, 'k*', markersize=15, label='Sensor')
axes[0, 0].set_xlabel('X position (m)')
axes[0, 0].set_ylabel('Y position (m)')
axes[0, 0].set_title('Particle Filter Tracking', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True)
axes[0, 0].axis('equal')

# Position error over time
pos_error = np.linalg.norm(x_est_pf[:, :2] - x_true[:, :2], axis=1)
axes[0, 1].plot(t_sim, pos_error, 'r-', linewidth=2)
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Position Error (m)')
axes[0, 1].set_title('Tracking Error', fontweight='bold')
axes[0, 1].grid(True)

# X position
axes[1, 0].plot(t_sim, x_true[:, 0], 'b-', linewidth=2, label='True')
axes[1, 0].plot(t_sim, x_est_pf[:, 0], 'r--', linewidth=2, label='PF estimate')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('X position (m)')
axes[1, 0].set_title('X Position Tracking', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Y position
axes[1, 1].plot(t_sim, x_true[:, 1], 'b-', linewidth=2, label='True')
axes[1, 1].plot(t_sim, x_est_pf[:, 1], 'r--', linewidth=2, label='PF estimate')
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Y position (m)')
axes[1, 1].set_title('Y Position Tracking', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

## 6. Visualize Particle Distribution

Show how particles represent the probability distribution at different times.

In [ ]:
# Visualize particles at specific timesteps
timesteps_to_show = [0, N//4, N//2, -1]
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, k in enumerate(timesteps_to_show):
    particles = particles_history[k]
    weights = weights_history[k]
    
    # Plot particles (size proportional to weight)
    axes[idx].scatter(particles[:, 0], particles[:, 1], 
                     s=weights * 1000, alpha=0.3, c='blue')
    
    # Plot true position
    axes[idx].plot(x_true[k, 0], x_true[k, 1], 'r*', 
                  markersize=20, label='True position')
    
    # Plot estimate
    axes[idx].plot(x_est_pf[k, 0], x_est_pf[k, 1], 'gx', 
                  markersize=15, markeredgewidth=3, label='PF estimate')
    
    # Plot sensor
    axes[idx].plot(0, 0, 'k*', markersize=15, label='Sensor')
    
    axes[idx].set_xlabel('X position (m)')
    axes[idx].set_ylabel('Y position (m)')
    axes[idx].set_title(f't = {t_sim[k]:.1f}s', fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_xlim([-8, 8])
    axes[idx].set_ylim([-8, 8])

plt.suptitle('Particle Distribution Evolution', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\nParticle cloud converges from initial uncertainty to accurate tracking!")

## 7. Compare Resampling Methods

Compare systematic, stratified, and residual resampling.

In [ ]:
resample_methods = ['systematic', 'stratified', 'residual']
results = {}

for method in resample_methods:
    # Reset particles
    particles_weights = (particles_init.copy(), weights_init.copy())
    x_est_method = []
    
    for k in range(N):
        particles_weights = PF.f(
            particles_weights=particles_weights,
            yk=y_meas[k:k+1],
            f=f_dynamics,
            f_params={},
            likelihood=likelihood,
            likelihood_params={},
            resample_method=method,
            resample_threshold=0.5
        )
        x_est_method.append(PF.h(particles_weights))
    
    results[method] = np.array(x_est_method)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Trajectory
axes[0].plot(x_true[:, 0], x_true[:, 1], 'k-', linewidth=2, label='True')
colors = ['red', 'blue', 'green']
for i, method in enumerate(resample_methods):
    axes[0].plot(results[method][:, 0], results[method][:, 1], 
                '--', color=colors[i], linewidth=2, label=f'{method.capitalize()}')
axes[0].set_xlabel('X position (m)')
axes[0].set_ylabel('Y position (m)')
axes[0].set_title('Resampling Method Comparison', fontweight='bold')
axes[0].legend()
axes[0].grid(True)
axes[0].axis('equal')

# Position error
for i, method in enumerate(resample_methods):
    error = np.linalg.norm(results[method][:, :2] - x_true[:, :2], axis=1)
    axes[1].plot(t_sim, error, color=colors[i], linewidth=2, label=f'{method.capitalize()}')
axes[1].set_xlabel('Time (s)')
axes[1].set_ylabel('Position Error (m)')
axes[1].set_title('Error Comparison', fontweight='bold')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Print statistics
print("\n=== Resampling Method Performance ===")
for method in resample_methods:
    error = np.linalg.norm(results[method][:, :2] - x_true[:, :2], axis=1)
    print(f"{method.capitalize():12s}: Mean error = {error.mean():.3f} m, Max error = {error.max():.3f} m")

## 8. Using Particle Filter with DynamicalSystem

Integrate the particle filter into pykal's compositional framework.

In [ ]:
# Create particle filter as a DynamicalSystem
pf_system = DynamicalSystem(
    f=PF.f,
    h=PF.h,
    state_name='particles_weights'
)

# Initialize parameters
param_dict = {
    'particles_weights': (particles_init.copy(), weights_init.copy()),
    'yk': np.array([0.0]),
    'f': f_dynamics,
    'f_params': {},
    'likelihood': likelihood,
    'likelihood_params': {},
    'resample_method': 'systematic',
    'resample_threshold': 0.5
}

# Run using DynamicalSystem interface
x_est_ds = []
for k in range(N):
    param_dict['yk'] = y_meas[k:k+1]
    new_state, x_est = pf_system.step(param_dict=param_dict, return_state=True)
    param_dict['particles_weights'] = new_state
    x_est_ds.append(x_est)

x_est_ds = np.array(x_est_ds)

print("\nParticle Filter integrated with DynamicalSystem:")
print(f"Final position: {x_est_ds[-1, :2]}")
print(f"True position: {x_true[-1, :2]}")
print(f"Error: {np.linalg.norm(x_est_ds[-1, :2] - x_true[-1, :2]):.3f} m")

## Conclusion

### Key Takeaways

1. **Particle filters excel at**:
   - Highly nonlinear problems (bearing-only tracking)
   - Non-Gaussian noise
   - Multi-modal distributions

2. **Resampling is crucial**:
   - Prevents particle degeneracy
   - Systematic resampling is generally recommended
   - Effective sample size (ESS) determines when to resample

3. **Trade-offs**:
   - More particles → Better accuracy, higher computational cost
   - Fewer particles → Faster, but risk of particle depletion

### When to Choose Particle Filter over UKF/EKF

- **Use PF** when: Severe nonlinearity, non-Gaussian noise, multi-hypothesis tracking
- **Use UKF** when: Moderate nonlinearity, Gaussian noise, computational efficiency matters
- **Use EKF** when: Mild nonlinearity, Gaussian noise, maximum speed required

### Next Steps

- See `pf_turtlebot.ipynb` for robot localization example
- See `pf_crazyflie.ipynb` for aerial vehicle tracking
- Experiment with different particle counts and resampling strategies

### Reference

Gordon, N. J., Salmond, D. J., & Smith, A. F. M. (1993). *Novel approach to nonlinear/non-Gaussian Bayesian state estimation*. IEE Proceedings F (Radar and Signal Processing), 140(2), 107-113.